In [1]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce

def mapMethytilatedCitosine(metfile,intersection_file_path,output_file_path):
    chrs_data = {}
    with gzip.open(metfile, 'rt') as f:
        for line in f:
            chr_index = line.split('\t')[0]
            coordinate_index = int(line.split('\t')[1])
            try:
                chrs_data[chr_index][coordinate_index] = line[:-5]
            except:
                chrs_data[chr_index] = {coordinate_index:line[:-5]}

    with open(intersection_file_path) as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)

    with open(output_file_path, 'a+') as whateverBox:
        for each_box in data:
            box_areas = {}
            box_start = int(each_box[1])
            box_end = int(each_box[2])
            chr_location = each_box[0]
            sequence_identifier = each_box[3]
            # tenemos que aplicar rango desde una posicion inicial porque es un archivo 0 based, si fuese 1 based el rango estaría mal y tendria que restarle
            # uno al archivo al box start
            box_areas['box'] = list(range(box_start, box_end))

            for boxarea in box_areas:
                for coordinate in box_areas[boxarea]:
                    matchline = chrs_data[chr_location].get(coordinate, False)
                    if matchline:
                        whateverBox.write('{}\t{}\t{}\n'.format(matchline, sequence_identifier, boxarea))
                        

def loadFileRemoveMtsAndLowMetReport(replicate):
        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', 'mets', 'nomets', 'context', 'boxID', 'side'],
            usecols=['chr', 'position', 'strand', 'boxID','mets', 'nomets', 'context']
                )
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        bxdf = bxdf[bxdf['mets'] + bxdf['nomets'] >= 4]
        return bxdf

def calculateValueAsPercentageOfMetCs(replicatedf,replicanamedf):
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf
def calculateValueAsCualitative(replicatedf,replicanamedf):
    """only zero or one, meaning one that there is methylation"""
    replicatedf[replicanamedf] = npwhere(replicatedf['mets']>replicatedf['nomets'], 1, 0)
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf

In [ ]:
import shutil
def movetofolderallthefiles(origin, destinationfolder):
    shutil.copy2(origin, destinationfolder)

In [2]:
working_folder = '/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps'
intersection_file = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'
basedmrfolder = '/home/joaquin/projects/methylation/data/DMRs'
experimentsClasification = {}
for root, dirs, files in os.walk(working_folder):

    for file in files:
        if 'CX_report.txt.gz' in file:
            replicate, hour, condition =root.strip().split('/')[-3:]
            experimentName = hour+condition
            experimentsClasification.setdefault(experimentName, [])
            targetMetFilename = root+'/'+file
            finalnamedestination = basedmrfolder+'/'+experimentName+replicate+'_CX_report.txt.gz'
            outputfile = targetMetFilename[:-3] + 'peakMets.tsv'
            experimentsClasification[experimentName].append(outputfile)
#             print(outputfile)
#             mapMethytilatedCitosine(targetMetFilename,intersection_file,outputfile)

            

In [3]:
allthedata = []
for experimentCondition in experimentsClasification:
    listOfDfReplicatesPercentageOfMetCs = []
    listOfDfReplicatesCualitative = []
    listOfNamesReplicates = []
    for replicatePath in experimentsClasification[experimentCondition]:

        replicaname =  experimentCondition+replicatePath.split('/')[-4]
        replicateDf = loadFileRemoveMtsAndLowMetReport(replicatePath)

        replicateDfPercentageOfMetCs = calculateValueAsPercentageOfMetCs(replicateDf,replicaname)

#         replicateDfCualitative = calculateValueAsCualitative(replicateDf,replicaname)
#         print(replicateDfCualitative)
        listOfNamesReplicates.append(replicaname)
        listOfDfReplicatesPercentageOfMetCs.append(replicateDfPercentageOfMetCs)
#         listOfDfReplicatesCualitative.append(replicateDfCualitative)
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), listOfDfReplicatesPercentageOfMetCs).fillna(0)
    df_merged[experimentCondition] = df_merged[listOfNamesReplicates].mean(axis=1)
    df_merged = df_merged.drop(columns=listOfNamesReplicates)
    allthedata.append(df_merged)
finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), allthedata).fillna(0)

In [ ]:
finaldf.to_csv('bisulfite.tsv',sep='\t', index=False)

In [ ]:
finaldf.to_csv('bisulfite.tsv',sep='\t', index=False)
CGdf = finaldf[(finaldf['context'] == 'CG')]

In [ ]:

finaldf = finaldf[['chr','position','strand','context','boxID','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]


In [ ]:
finaldf

In [ ]:

CGdf = finaldf[(finaldf['context'] == 'CG')]
CHHdf = finaldf[(finaldf['context'] == 'CHH')]
CHGdf = finaldf[(finaldf['context'] == 'CHG')]

CGdf.to_csv('CG_BSMet.tsv',sep='\t', index=False)

CHGdf.to_csv('CHG_BSMet.tsv',sep='\t', index=False)

CHHdf.to_csv('CHH_BSMet.tsv',sep='\t', index=False)

In [ ]:
CGdf

In [ ]:
CGdf

In [ ]:
import pandas as pd
import seaborn as sns
import math
# CGdf = CGdf.drop(columns=['chr','position', 'strand','context','boxID'])
# CGdf = CGdf.set_index()

# 

In [ ]:
CGdf = CGdf[['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']]


In [ ]:
sns.heatmap(CGdf, cmap="mako")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,20))         # Sample figsize in inches
sns.heatmap(CGdf, cmap="tab10", ax=ax)
# sns.set_context("notebook", font_scale=1.75, rc={"lines.linewidth": 2.5, 'font.family':'Helvetica'})
# sns.heatmap(CGdf, cmap="tab10")

In [ ]:
Sinmenosde1 = CGdf[(CGdf >= 0.01).all(1)]
fig, ax = plt.subplots(figsize=(10,20)) # Sample figsize in inches
sns.heatmap(Sinmenosde1, cmap="tab10", ax=ax)


In [ ]:
sns.set_context("paper", font_scale=1.75, rc={"lines.linewidth": 2.5, 'font.family':'Helvetica'})
fig, ax = plt.subplots(figsize=(10,30))         # Sample figsize in inches
sns.heatmap(Sinmenosde1, cmap="mako", ax=ax)

In [ ]:
Sinmenosde1

In [ ]:
for col in ['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']:
    print(col, CGdf[col].mean()*100)

In [ ]:
CHHdf = finaldf[(finaldf['context'] == 'CHH')]
for col in ['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']:
    print(col, CHHdf[col].mean()*100)

In [ ]:
CHGdf = finaldf[(finaldf['context'] == 'CHG')]
for col in ['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']:
    print(col, CHGdf[col].mean()*100)

In [ ]:
def mergeResultsMets(folder):
    with cd('/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2/reports/'):
        file_names = os.listdir()
        for rep in ['rep1', 'rep2']:
            for side in ['.motif.', 'motifSide']:
                for metcontex in ['CG', 'CHG', 'CHH']:
                    for box in ['G_box', 'GAC_box', 'PBE_box', 'GCC_box', 'TG_box']:
                        boxfiles = []
                        for file in file_names:
                            # print(file_names)
                            # print('hello')
                            if rep in file and side in file and metcontex in file and file.startswith(box):
                                # print('hello')
                                if '_merged_intersect.csv' not in file:
                                    boxfiles.append(file)
                        # print(boxfiles)
                        
                        if box + rep + side + metcontex + '_merged_intersect.csv' in file_names:
                            experiment1df = pd.read_csv(box + rep + side + metcontex + '_merged_intersect.csv')
                            experiment2df = pd.read_csv(boxfiles[0])
                            mergedDf = experiment1df.merge(experiment2df, on=['chr', 'position', 'strand','boxID'], how='outer')
                            mergedDf.to_csv(box + rep + side + metcontex + '_merged_intersect.csv', index=False)
                            os.remove(boxfiles[0])
                        elif len(boxfiles) == 2:
                            # print('hello')
                            experiment1df = pd.read_csv(boxfiles[0])
                            experiment2df = pd.read_csv(boxfiles[1])
                            # print(experiment1df)
                            mergedDf = experiment1df.merge(experiment2df, on=['chr', 'position', 'strand','boxID'], how='outer')
                            mergedDf.to_csv(box + rep + side + metcontex + '_merged_intersect.csv', index=False)
                            os.remove(boxfiles[0])
                            os.remove(boxfiles[1])


In [ ]:
peaks

In [4]:
import pandas as pd 
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])
def getpositions(x):
    chrm, center = x.strip().split(':')
    return chrm, int(center)-100, int(center)+101
df  = finaldf[(finaldf['context'] != 'CHG')]
metadata = df.iloc[:,4:5]
values = df.iloc[:,5:]
values = values.loc[(values!=0).any(axis=1)]
merged = metadata.merge(values, left_index=True, right_index=True, how='inner')
meandf = merged.groupby('boxID', as_index=False).mean()
peaks = peaks.merge(meandf, on='boxID', how='outer')
peaks =peaks.fillna(0)

# peaks.to_csv('CG_BSMet_mean.tsv',sep='\t', index=False)

# peaks.to_csv('CHG_BSMet_mean.tsv',sep='\t', index=False)

peaks.to_csv('non-CG_BSMetmean.tsv',sep='\t', index=False)
# meandf['chr'], meandf['start'], meandf['end'] =  zip(*meandf['boxID'].map(lambda x: getpositions(x)))

In [ ]:
import pandas as pd 
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])
def getpositions(x):
    chrm, center = x.strip().split(':')
    return chrm, int(center)-100, int(center)+101
df  = pd.read_csv('CG_BSMet.tsv', sep='\t')


In [ ]:
df


In [ ]:
metadata = df.iloc[:,4:5]
values = df.iloc[:,5:]
values = values.loc[(values!=0).any(axis=1)]
merged = metadata.merge(values, left_index=True, right_index=True, how='inner')
meandf = merged.groupby('boxID', as_index=False).mean()
peaks = peaks.merge(meandf, on='boxID', how='outer')
peaks =peaks.fillna(0)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(null_data)
